In [23]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [25]:
URL = "https://www.madlan.co.il/for-sale/נוף-הגליל-ישראל"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

page = requests.get(URL, headers=headers)
if page.status_code == 200:
    soup = BeautifulSoup(page.content, "html.parser")
    apartments = []
    links = soup.find_all('a', href=True)
    for link in links:
        href = link['href']
        if href.startswith('/listings'):
            apartment_url = f"https://www.madlan.co.il{href}"
            print(apartment_url)
            apartment_page = requests.get(apartment_url, headers=headers)
            if apartment_page.status_code == 200:
                apartment_soup = BeautifulSoup(apartment_page.content, "html.parser")

                primary_address = apartment_soup.find('h1', {'data-auto': 'primary_address_text'})
                if primary_address is None:
                    continue

                address_parts = primary_address.text.split()
                if len(address_parts) < 2:
                    continue

                street = ' '.join(address_parts[:-1])
                number_in_street = ''.join(filter(str.isdigit, address_parts[-1]))
                if number_in_street:
                    number_in_street = int(number_in_street)
                else:
                    number_in_street = None

                secondary_address = apartment_soup.find('div', {'data-auto': 'secondary_address_text'}).text
                if ',' in secondary_address:
                    city_area, city = secondary_address.split(', ')
                else:
                    city_area = ''
                    city = secondary_address

                type_ = apartment_soup.find('div', {'data-auto': 'business-class'}).text
                room_number = apartment_soup.find('div', {'data-auto': 'beds-count'}).text

                floor_out_of = apartment_soup.find('div', {'data-auto': 'floor'}).text
                area = apartment_soup.find('div', {'data-auto': 'area'}).text

                condition = apartment_soup.find('div', {'data-auto': 'unit-general-condition-value'}).text
                entranceDate = apartment_soup.find('div', {'data-auto': 'unit-availability-value'}).text
                furniture = apartment_soup.find('div', {'data-auto': 'unit-furniture-details-value'}).text

                description_elem = apartment_soup.find('div', {'class': 'css-v1qjdi ebqee3y2'})
                description = description_elem.text if description_elem else ''

                price_text = apartment_soup.find('div', {'data-auto': 'current-price'}).text
                price = int(re.sub(r'\D', '', price_text))
                specification = apartment_soup.find('div', {'data-auto' : "amenities-block"}).get_text()
                hasElevator = 'יש מעלית' in specification
                hasParking = "יש חניה" in specification
                hasBars = "יש סורגים" in specification
                hasStorage = 'יש מחסן' in specification
                hasAirCondition = "יש מיזוג אויר" in specification
                hasBalcony = "יש מרפסת" in specification
                hasMamad = "יש ממ" in specification
                handicapFriendly = "לא נגיש לנכים" not in specification
                num_of_images =len(apartment_soup.find_all('div' ,{'data-auto':"bulletin-image"}))
                

                apartments.append({
                    'City': city,
                    'type': type_,
                    'room_number': room_number,
                    'Area': area,
                    'Street': street,
                    'number_in_street': number_in_street,
                    'city_area': city_area,
                    'price': price,
                    'floor_out_of': floor_out_of,
                    'condition': condition,
                    'entranceDate': entranceDate,
                    'furniture': furniture,
                    'description': description,
                    'hasElevator': hasElevator,
                    'hasParking' : hasParking,
                    'hasBars' : hasBars,
                    'hasStorage' : hasStorage,
                    'hasAirCondition' : hasAirCondition,
                    'hasBalcony' : hasBalcony,
                    'hasMamad' : hasMamad,
                    'handicapFriendly' : handicapFriendly,
                    'num_of_images': num_of_images

                })
            else:
                print(f'Error status code {apartment_page.status_code} link {apartment_url}')
                break
                

    df = pd.DataFrame(apartments)
    df
else:
    print(f'Error status code {page.status_code}')

https://www.madlan.co.il/listings/euhRflurwgL
https://www.madlan.co.il/listings/gMOKgPT2NHO
https://www.madlan.co.il/listings/cmxUkaAA5sg
https://www.madlan.co.il/listings/c0ACy9nZhel
https://www.madlan.co.il/listings/9TMTVDgBn5M
https://www.madlan.co.il/listings/VPUHvpd9Zwz
https://www.madlan.co.il/listings/N2cmBlFcfAQ
https://www.madlan.co.il/listings/ddPjHr2N42W
https://www.madlan.co.il/listings/d0LiaelgYhN
https://www.madlan.co.il/listings/esa5sc6sYrv
https://www.madlan.co.il/listings/gQ5WmUeVLB0
https://www.madlan.co.il/listings/gaIvmsIslNp
https://www.madlan.co.il/listings/hgQRxUVgcEH
Error status code 403 link https://www.madlan.co.il/listings/hgQRxUVgcEH


In [26]:
df.to_excel('output.xlsx')

In [27]:
df

,City,type,room_number,Area,Street,number_in_street,city_area,price,floor_out_of,condition,...,description,hasElevator,hasParking,hasBars,hasStorage,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,num_of_images
0,נוף הגליל,דירה,3 חד׳,"78 מ""ר",הרב פנחס מילר,1,הר יונה ג',985000,קומה 5 מתוך 8,חדש,...,"דירה יפה, חדשה, מקום נפלא אזור דתי חרדי, נוף מ...",False,False,False,False,True,False,False,False,0
1,נוף הגליל,דירה,3 חד׳,"70 מ""ר",דרך אריאל שרון,19,שלום,1030000,קומת קרקע,משופץ,...,"דירה שמורה עם שני כיווני אוויר, חזיתית לכיוון ...",False,False,True,False,True,False,False,False,0
2,נוף הגליל,דירה,4 חד׳,"91 מ""ר",הרב עובדיה יוסף,1,צפונית,1050000,קומה 1 מתוך 3,משופץ,...,דירה ממש יפה ברחוב הרב עובדיה יוסף 1 *דירה ממש...,True,False,False,True,True,False,True,False,0
3,נוף הגליל,דירה,2 חד׳,"45 מ""ר",מנחם אריאב,27,צפונית,660000,קומה 1 מתוך 3,משופץ,...,בית מסודר יפה ושמור .. ללא טיווח . משקיעים בבק...,False,False,True,False,True,False,True,False,0
4,נוף הגליל,דירה,4 חד׳,"106 מ""ר",חרוד,16,אשכול,1350000,קומה 7 מתוך 16,משופץ,...,"פרטי הריהוט:\n4 מזגנים, מטבח מאובזר כולל מדיח ...",True,False,True,False,True,False,True,True,10
5,נוף הגליל,דירה,4 חד׳,"106 מ""ר",חרוד,16,אשכול,1230000,קומה 5 מתוך 16,משופץ,...,"דירה מקסימה, מוארת ומסודרת בבניין מתחדש *להיכנ...",True,False,False,True,True,False,False,False,9
6,נוף הגליל,דירה,3 חד׳,"74 מ""ר",דרך אריאל שרון,19,שלום,950000,קומה 2 מתוך 4,שמור,...,,False,False,False,False,False,False,False,False,4
7,נוף הגליל,דירה,4 חד׳,"100 מ""ר",אלומה,6,הר יונה,1530000,קומה 2 מתוך 3,משופץ,...,לא מתיווך! לא לתיווך! מתווכים לא להתקשר!\nדירה...,False,False,False,True,True,True,True,False,4
8,נוף הגליל,דירה,4 חד׳,"94 מ""ר",הרב עובדיה יוסף,1,צפונית,1200000,קומת קרקע,חדש,...,הנוף הוא חלומי פשוט לראות ולחטוף,True,False,True,False,True,False,True,False,0
9,נוף הגליל,דירה,3 חד׳,"80 מ""ר",הרב עובדיה יוסף,1,צפונית,1100000,קומה 4 מתוך 6,חדש מקבלן,...,מיקום חרדי ממש בנין חדש מהקבלן לתמונות הנכס ני...,True,False,True,False,True,False,True,False,0
